# Annotations Implementation

## Setup

In [ ]:
import os
import sys

main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(main_dir)
print(main_dir)

import imageio
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from MambaFCS.changedetection.datasets.make_data_loader import SemanticChangeDetectionDatset, make_data_loader, SemanticChangeDetectionDatset_LandSat
from MambaFCS.changedetection.utils_func.metrics import Evaluator
from MambaFCS.changedetection.models.STMambaSCD import STMambaSCD
import MambaFCS.changedetection.utils_func.lovasz_loss as L
from torch.optim.lr_scheduler import StepLR
from MambaFCS.changedetection.utils_func.mcd_utils import accuracy, SCDD_eval_all, AverageMeter
from MambaFCS.changedetection.utils_func.loss import contrastive_loss, ce2_dice1, ce2_dice1_multiclass, SEK_loss_from_eval
from MambaFCS.changedetection.configs.config import get_config
import MambaFCS.changedetection.datasets.imutils as imutils
import torchvision.transforms.functional as TF

from dotenv import load_dotenv

load_dotenv()
def getPath(env_path):
    return os.path.expanduser(os.getenv(env_path))


torch.cuda.set_device(0)

/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/classification/models/vmamba.py:252: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/classification/models/vmamba.py:260: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/classification/models/vmamba.py:275: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is 

In [2]:
VSSM_MODEL_PATH = getPath('VSSMBASEPATH')

LandSat_DATASET_PATH = getPath('LANDSAT')
print(LandSat_DATASET_PATH)
LandSat_TRAIN_DATA_LIST_PATH = os.path.join(LandSat_DATASET_PATH, 'train_list.txt')

LandSat_TEST_DATA_LIST_PATH = os.path.join(LandSat_DATASET_PATH, 'test_list.txt')

LandSat_VAL_DATA_LIST_PATH = os.path.join(LandSat_DATASET_PATH, 'val_list.txt')

configs_path = os.path.join(main_dir, 'MambaFCS/changedetection/configs/vssm1/vssm_base_224.yaml')

model_path = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/JOURNAL/')

MAMBAFCS = os.path.abspath('/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/saved_models/LandSat_SCD_with_DICE/75000_model_0.603.pth')

SAVE_IMGS_PATH = os.path.join(main_dir, 'MambaFCS/annotations/Images/Landsat/')

train_data_list = []
with open(LandSat_TRAIN_DATA_LIST_PATH, 'r') as f:
    for line in f:
        train_data_list.append(line.strip())

with open(LandSat_VAL_DATA_LIST_PATH, 'r') as f:
    for line in f:
        train_data_list.append(line.strip())


test_data_list = []
with open(LandSat_TEST_DATA_LIST_PATH, 'r') as f:
    for line in f:
        test_data_list.append(line.strip())

NUM_CLASSES = 5

/storage/scratch3/buddhiw-change-detection/Datasets/Landsat-SCD/


In [3]:
class ARGS:
    def __init__(self):
        self.cfg = configs_path
        self.opts = None
        self.pretrained_weight_path = VSSM_MODEL_PATH
        self.dataset = 'Landsat'
        self.type = 'train'
        self.train_dataset_path = LandSat_TRAIN_DATA_LIST_PATH

        self.test_dataset_path = LandSat_TEST_DATA_LIST_PATH

        
        self.shuffle = True
        self.batch_size = 4
        self.crop_size = 512
        self.train_data_name_list = train_data_list
        self.test_data_name_list = test_data_list
        self.start_iter = 0
        self.cuda = True
        self.max_iters = 800000
        self.model_type = 'MambaSCD_base'
        self.model_param_path = model_path

        self.resume = MAMBAFCS

        self.learning_rate = 1e-4
        self.momentum = 0.9
        self.weight_decay = 5e-4
        self.num_classes = 5

args = ARGS()

In [4]:
config = get_config(args)

deep_model = STMambaSCD(
    output_cd = 2, 
    output_clf = args.num_classes,
    pretrained=args.pretrained_weight_path,
    patch_size=config.MODEL.VSSM.PATCH_SIZE, 
    in_chans=config.MODEL.VSSM.IN_CHANS, 
    num_classes=config.MODEL.NUM_CLASSES, 
    depths=config.MODEL.VSSM.DEPTHS, 
    dims=config.MODEL.VSSM.EMBED_DIM, 
    # ===================
    ssm_d_state=config.MODEL.VSSM.SSM_D_STATE,
    ssm_ratio=config.MODEL.VSSM.SSM_RATIO,
    ssm_rank_ratio=config.MODEL.VSSM.SSM_RANK_RATIO,
    ssm_dt_rank=("auto" if config.MODEL.VSSM.SSM_DT_RANK == "auto" else int(config.MODEL.VSSM.SSM_DT_RANK)),
    ssm_act_layer=config.MODEL.VSSM.SSM_ACT_LAYER,
    ssm_conv=config.MODEL.VSSM.SSM_CONV,
    ssm_conv_bias=config.MODEL.VSSM.SSM_CONV_BIAS,
    ssm_drop_rate=config.MODEL.VSSM.SSM_DROP_RATE,
    ssm_init=config.MODEL.VSSM.SSM_INIT,
    forward_type=config.MODEL.VSSM.SSM_FORWARDTYPE,
    # ===================
    mlp_ratio=config.MODEL.VSSM.MLP_RATIO,
    mlp_act_layer=config.MODEL.VSSM.MLP_ACT_LAYER,
    mlp_drop_rate=config.MODEL.VSSM.MLP_DROP_RATE,
    # ===================
    drop_path_rate=config.MODEL.DROP_PATH_RATE,
    patch_norm=config.MODEL.VSSM.PATCH_NORM,
    norm_layer=config.MODEL.VSSM.NORM_LAYER,
    downsample_version=config.MODEL.VSSM.DOWNSAMPLE,
    patchembed_version=config.MODEL.VSSM.PATCHEMBED,
    gmlp=config.MODEL.VSSM.GMLP,
    use_checkpoint=config.TRAIN.USE_CHECKPOINT,
    ) 

deep_model = deep_model.cuda()

=> merge config from /storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/MambaFCS/changedetection/configs/vssm1/vssm_base_224.yaml
Successfully load ckpt /storage/scratch3/buddhiw-change-detection/ChangeDetection/VSSModels/pretrained/vssm_base_0229_ckpt_epoch_237.pth
_IncompatibleKeys(missing_keys=['outnorm0.weight', 'outnorm0.bias', 'outnorm1.weight', 'outnorm1.bias', 'outnorm2.weight', 'outnorm2.bias', 'outnorm3.weight', 'outnorm3.bias'], unexpected_keys=['classifier.norm.weight', 'classifier.norm.bias', 'classifier.head.weight', 'classifier.head.bias'])
False


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 1 has a total capacity of 47.52 GiB of which 4.38 MiB is free. Process 2911810 has 15.66 GiB memory in use. Process 2764986 has 15.55 GiB memory in use. Process 444288 has 7.17 GiB memory in use. Process 2816589 has 7.17 GiB memory in use. Process 930308 has 1.56 GiB memory in use. Including non-PyTorch memory, this process has 358.00 MiB memory in use. Of the allocated memory 95.51 MiB is allocated by PyTorch, and 2.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
deep_model.load_state_dict(torch.load(args.resume))


deep_model.cuda()
deep_model.eval()

STMambaSCD(
  (encoder): Backbone_VSSM(
    (patch_embed): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): Permute()
      (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (3): Permute()
      (4): GELU(approximate='none')
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (6): Permute()
      (7): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (layers): ModuleList(
      (0): Sequential(
        (blocks): Sequential(
          (0): VSSBlock(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (op): SS2D(
              (out_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (in_proj): Linear(in_features=128, out_features=256, bias=False)
              (act): SiLU()
              (conv2d): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=256, bias=False)
              (out_proj): Linear(in_

In [ ]:
from thop import profile, clever_format

# Create dummy input matching your model's input shape
dummy_pre = torch.randn(1, 3, 512, 512).cuda()
dummy_post = torch.randn(1, 3, 512, 512).cuda()

macs, params = profile(deep_model, inputs=(dummy_pre, dummy_post))
macs, params = clever_format([macs, params], "%.3f")

print(f"MACs: {macs}")
print(f"Parameters: {params}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.AdaptiveMaxPool2d'>.
MACs: 263.157G
Parameters: 189.545M


In [ ]:
def img_loader(path):
    img = np.array(imageio.imread(path), np.float32)
    return img

In [ ]:
from pathlib import Path
import numpy as np
import imageio.v3 as imageio
import torch
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from tqdm import tqdm
import os


def plot_transition_matrix(t1_labels, t2_labels,
                           class_names, class_ids,
                           title="",
                           save_path=None,
                           cmap="Blues",
                           data_override=None):

    if data_override is None:
        id_to_idx = {cid: idx for idx, cid in enumerate(class_ids)}
        n_cls     = len(class_ids)
        trans_mat = np.zeros((n_cls, n_cls), dtype=np.int64)

        t1_flat, t2_flat = t1_labels.ravel(), t2_labels.ravel()
        mask             = t1_flat != t2_flat
        for f, t in zip(t1_flat[mask], t2_flat[mask]):
            if f in id_to_idx and t in id_to_idx:
                trans_mat[id_to_idx[f], id_to_idx[t]] += 1
    else:
        trans_mat = data_override
        n_cls     = trans_mat.shape[0]

    tot   = trans_mat.sum()
    pct   = np.where(tot == 0, 0.0, trans_mat / tot * 100)

    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(pct, cmap=cmap, vmin=0, vmax=pct.max())

    ax.set_xticks(range(n_cls)); ax.set_yticks(range(n_cls))
    ax.set_xticklabels(class_names, rotation=25)
    ax.set_yticklabels(class_names)

    ax.xaxis.set_label_position("top"); ax.xaxis.tick_top()
    ax.tick_params(axis="x", labelbottom=False)
    ax.set_xlabel("To"); ax.set_ylabel("From"); ax.set_title(title, pad=10)

    for i in range(n_cls):
        for j in range(n_cls):
            ax.text(j, i, f"{pct[i,j]:.2f}%",
                    ha="center", va="center",
                    color="white" if pct[i,j] > pct.max()*0.6 else "black",
                    fontsize=8)

    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
    else:
        plt.show()


def compute_dataset_transition_matrices(test_imgs,
                                        deep_model,
                                        class_ids   = [1, 2, 3, 4, 5, 6],
                                        class_names = ['low vegetation',
                                                       'ground',
                                                       'tree',
                                                       'water',
                                                       'Building',
                                                       'Playground'],
                                        device      = "cuda",
                                        save_dir    = "."):
    

    Dataset_Path = LandSat_DATASET_PATH
    test_imgs_A = [os.path.join(Dataset_Path, 'A', img) for img in test_imgs]
    test_imgs_B = [os.path.join(Dataset_Path, 'B', img) for img in test_imgs]
    test_imgs_label_A = [os.path.join(Dataset_Path, 'labelA', img) for img in test_imgs]
    test_imgs_label_B = [os.path.join(Dataset_Path, 'labelB', img) for img in test_imgs]

    id_to_idx  = {cid: idx for idx, cid in enumerate(class_ids)}
    n_cls      = len(class_ids)

    gt_counts   = np.zeros((n_cls, n_cls), dtype=np.int64)
    pred_counts = np.zeros((n_cls, n_cls), dtype=np.int64)

    def add_counts(mat: np.ndarray, a: np.ndarray, b: np.ndarray):
        flat_a, flat_b = a.ravel(), b.ravel()
        for f, t in zip(flat_a, flat_b):
            if f in id_to_idx and t in id_to_idx:
                mat[id_to_idx[f], id_to_idx[t]] += 1

    deep_model.eval().to(device)

    for img_A_path, img_B_path, label_A_path, label_B_path in tqdm(
            zip(test_imgs_A, test_imgs_B, test_imgs_label_A, test_imgs_label_B), total=len(test_imgs_A)):
        name = os.path.basename(img_A_path)

        gt_t1 = imageio.imread(label_A_path)
        gt_t2 = imageio.imread(label_B_path)
        add_counts(gt_counts, gt_t1, gt_t2)

        pre_image  = img_loader(img_A_path)
        post_image = img_loader(img_B_path)

        pre_tensor = TF.to_tensor(imutils.normalize_img(pre_image)).unsqueeze(0).cuda()
        post_tensor = TF.to_tensor(imutils.normalize_img(post_image)).unsqueeze(0).cuda()

        with torch.no_grad():
            output_cd, output_t1, output_t2 = deep_model(pre_tensor, post_tensor)

            change_mask = torch.argmax(output_cd, axis=1)

            preds_A = torch.argmax(output_t1, dim = 1)
            preds_B = torch.argmax(output_t2, dim = 1)

            preds_A = (preds_A*change_mask.squeeze().long()).cpu().numpy()
            preds_B = (preds_B*change_mask.squeeze().long()).cpu().numpy()
            

        add_counts(pred_counts, preds_A, preds_B)

    plot_transition_matrix(None, None,
                           class_names=class_names,
                           class_ids=class_ids,
                        #    title="GT  transitions – whole test set",
                           save_path=Path(save_dir) / "GT_transition_matrix_landsat.png",
                           data_override=gt_counts)

    plot_transition_matrix(None, None,
                           class_names=class_names,
                           class_ids=class_ids,
                        #    title="Predicted transitions – whole test set",
                           save_path=Path(save_dir) / "Pred_transition_matrix_landsat.png",
                           data_override=pred_counts)

    diff_counts = pred_counts - gt_counts

    plot_transition_matrix(None, None,
                           class_names=class_names,
                           class_ids=class_ids,
                        #    title="Difference (Pred - GT) transitions – whole test set",
                           save_path=Path(save_dir) / "Diff_transition_matrix_landsat.png",
                           data_override=diff_counts)   

    return gt_counts, pred_counts


In [ ]:
full_images = test_data_list

compute_dataset_transition_matrices(test_imgs=full_images, deep_model = deep_model, class_ids=[1, 2, 3, 4],class_names=[
    'Farmland', 'Desert', 'Building', 'Water'
]) 

100%|██████████| 477/477 [02:34<00:00,  3.09it/s]


(array([[      0, 1365749,  664320,       0],
        [9330980,       0,  573561, 1787659],
        [ 257248,   79125,       0,       0],
        [  67628,  828049,       0,       0]]),
 array([[  19353, 1250450,  660065,     300],
        [9444702,   20337,  574745, 1793702],
        [ 256970,   76086,    4908,       3],
        [  68741,  822982,     410,    1454]]))

# Annotations for the LandSat dataset

In [ ]:
ori_label_value_dict = {
    'No change': (255, 255, 255),
    'Farmland': (0, 155, 0),
    'Desert': (255, 165, 0),
    'Building': (230, 30, 100),
    'Water': (0, 170, 240)
}

target_label_value_dict = {
    'No change': 0,
    'Farmland': 1,
    'Desert': 2,
    'Building': 3,
    'Water': 4
}

In [ ]:
def annotate_image_LANDSAT(image_number):
    pre_image_path = os.path.join(LandSat_DATASET_PATH,"A",f"{image_number}")
    post_image_path = os.path.join(LandSat_DATASET_PATH,"B",f"{image_number}")
    GT_T1_image_path = os.path.join(LandSat_DATASET_PATH,"labelA_rgb",f"{image_number}")
    GT_T2_image_path = os.path.join(LandSat_DATASET_PATH,"labelB_rgb",f"{image_number}")

    image_number = image_number[:-4] 

    pre_image = img_loader(pre_image_path)
    post_image = img_loader(post_image_path)
    GT_T1_image = img_loader(GT_T1_image_path)
    GT_T2_image = img_loader(GT_T2_image_path)
    
    pre_img = imutils.normalize_img(pre_image)  # imagenet normalization
    pre_img = np.transpose(pre_img, (2, 0, 1))

    post_img = imutils.normalize_img(post_image)  # imagenet normalization
    post_img = np.transpose(post_img, (2, 0, 1))

    GT_T1_image = np.asarray(GT_T1_image)
    GT_T2_image = np.asarray(GT_T2_image)

    # save original images
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pre.png"), pre_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_post.png"), post_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_gt_t1.png"), GT_T1_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_gt_t2.png"), GT_T2_image.astype(np.uint8))

    pre_tensor = TF.to_tensor(imutils.normalize_img(pre_image)).unsqueeze(0).cuda()
    post_tensor = TF.to_tensor(imutils.normalize_img(post_image)).unsqueeze(0).cuda()
    label_t1 = TF.to_tensor(GT_T1_image).cuda().long()
    label_t2 = TF.to_tensor(GT_T2_image).cuda().long()

    with torch.no_grad():
        output_cd, output_t1, output_t2 = deep_model(pre_tensor, post_tensor)

        change_mask = torch.argmax(output_cd, axis=1)

        preds_A = torch.argmax(output_t1, dim = 1)
        preds_B = torch.argmax(output_t2, dim = 1)

        preds_A = (preds_A*change_mask.squeeze().long()).cpu().numpy()
        preds_B = (preds_B*change_mask.squeeze().long()).cpu().numpy()

    preds_A = map_labels_to_colors(np.squeeze(preds_A), ori_label_value_dict=ori_label_value_dict, target_label_value_dict=target_label_value_dict)
    preds_B = map_labels_to_colors(np.squeeze(preds_B), ori_label_value_dict=ori_label_value_dict, target_label_value_dict=target_label_value_dict)


    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pred_t1.png"), preds_A.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pred_t2.png"), preds_B.astype(np.uint8))

In [ ]:
# annotate_image_LANDSAT("From1997To2017_12.png")